In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

test_data = pd.read_csv(r"D:\Downloads\Germany_Volatile_data_graph.csv")
predictions = pd.read_csv(r"D:\Downloads\Germany_Volatile_predicted_data.csv")

test_data['Datetime'] = pd.to_datetime(test_data['Datetime'])
predictions['Datetime'] = pd.to_datetime(predictions['Datetime'])


data = test_data.merge(
    predictions, 
    on='Datetime', 
    suffixes=('_actual', '_predicted')
)

data.rename(columns={'Price': 'market_price', 'Predicted': 'market_price_predicted'}, inplace=True)

data['market_price'] /= 1000
data['market_price_predicted'] /= 1000

battery_capacity = 55  
starting_soc = 50 
initial_budget = 100  
charge_efficiency = 0.95
discharge_efficiency = 0.95


actions = ["Charge", "Discharge", "Hold"]

features = []
labels = []

soc = starting_soc
for t in range(len(data) - 1):
    current_price = data['market_price'].iloc[t]
    predicted_future_price = data['market_price_predicted'].iloc[t + 1]
    
    soc_kwh = (soc / 100) * battery_capacity  
    
    features.append([soc, current_price, predicted_future_price])

    if soc < 100 and current_price < predicted_future_price:  
        labels.append(0)
    elif soc > 0 and current_price > predicted_future_price: 
        labels.append(1) 
    else:
        labels.append(2) 


features = np.array(features)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_pred))

charge_indices, discharge_indices = [], []
soc = starting_soc
budget = initial_budget
cumulative_profits, soc_values, decisions = [], [soc], []

for t in range(len(data) - 1):
    current_price = data['market_price'].iloc[t]
    predicted_future_price = data['market_price_predicted'].iloc[t + 1]
    
    feature = np.array([[soc, current_price, predicted_future_price]])

    action_idx = rf_model.predict(feature)[0]
    action = actions[action_idx]
    decisions.append(action)
    
    soc_kwh = (soc / 100) * battery_capacity
    if action == "Charge" and soc < 100:
        amount = min(battery_capacity - soc_kwh, 10)  
        cost = current_price * amount / charge_efficiency
        budget -= cost 
        soc += (amount / battery_capacity) * 100 
        charge_indices.append(t)
    elif action == "Discharge" and soc > 0:
        amount = min(soc_kwh, 10)  
        revenue = predicted_future_price * amount * discharge_efficiency
        budget += revenue
        soc -= (amount / battery_capacity) * 100  
        discharge_indices.append(t)
    
    soc = max(0, min(100, soc))  
    cumulative_profits.append(budget - initial_budget)
    soc_values.append(soc)


plt.figure(figsize=(12, 6))
x = range(len(data))
plt.plot(x, data["market_price"], label="Market Price (EUR/kWh)", color='orange', linestyle="--")
plt.scatter(charge_indices, data["market_price"].iloc[charge_indices], color="green", label="Charge Actions")
plt.scatter(discharge_indices, data["market_price"].iloc[discharge_indices], color="red", label="Discharge Actions")
plt.plot(x, data["market_price_predicted"], label="Predicted Price (EUR/kWh)", color="blue")
plt.xlabel("Hours")
plt.ylabel("Price (€/kWh)")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(x[:-1], cumulative_profits, label="Cumulative Profit (€)", color="orange")
plt.axhline(0, color="red", linestyle="--", label="Break-even")
plt.xlabel("Hours")
plt.ylabel("Profit (€)")
plt.legend()
plt.grid()
plt.title("Cumulative Profits Over Time")
plt.show()

print("Action counts:", {action: decisions.count(action) for action in actions})
print(f"Final budget: €{budget:.2f}")
print(f"Net profit: €{budget - initial_budget:.2f}")